In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import pytorch_lightning
from nlp_datasets import get_dataloader, get_dataset

import torch
from torch import optim
from torch.optim.lr_scheduler import StepLR
from tensorboardX import SummaryWriter
from accelerate import Accelerator
from safetensors.torch import load_file

import json
import gc

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
writer = SummaryWriter()
accelerator = Accelerator()

In [3]:
torch.cuda.empty_cache()
gc.collect()

60

In [4]:
text_decoder_id = f"MLP-KTLim/llama-3-Korean-Bllossom-8B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(
    text_decoder_id,
    lagacy = False
)

tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
base_model = AutoModelForCausalLM.from_pretrained(
    text_decoder_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True, 
    quantization_config=bnb_config
    )

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
for name, module in base_model.named_modules():
    print(name)
    break

In [7]:
train_dataloader = get_dataloader()
valid_dataloader = get_dataloader(split='valid')

dataset_dict:  {'prompt': '\n        system : 한국말로만 대답하고 최대한 간결하고 알기쉽게 정리해줘.\n        user : 사건의 title과 판시사항을 보고 판결 결과와 그 이유를 예측해줘\n        \n title:[손해배상(기)]〈일제 강제동원 피해자들이 일본 기업을 상대로 불법행위로 인한 위자료 지급을 구하는 사건〉 [공2024상,204]\n판시사항:[1] 객관적으로 채권자가 권리를 행사할 수 없는 장애사유가 있었던 경우, 채무자가 소멸시효 완성을 주장하는 것이 신의성실의 원칙에 반하는 권리남용에 해당하는지 여부(적극)\n[2] 채권자에게 권리의 행사를 기대할 수 없는 객관적인 사실상의 장애사유가 있었으나 대법원이 채권자의 권리행사가 가능하다는 법률적 판단을 내린 경우, 그 시점 이후에는 장애사유가 해소되었다고 볼 수 있는지 여부(원칙적 적극)\n[3] 일제강점기에 강제동원되어 기간 군수사업체인 구 미쓰비시중공업 주식회사에서 강제노동에 종사한 갑 등이 위 회사가 해산된 후 새로이 설립된 미쓰비시중공업 주식회사를 상대로 위자료 지급을 구한 사안에서, 강제동원 피해자의 일본 기업에 대한 위자료청구권은 ‘대한민국과 일본국 간의 재산 및 청구권에 관한 문제의 해결과 경제협력에 관한 협정’의 적용 대상에 포함되지 않는다는 법적 견해를 최종적으로 명확하게 밝힌 대법원 2018. 10. 30. 선고 2013다61381 전원합의체 판결이 선고될 때까지는 강제동원 피해자 또는 그 상속인들에게는 미쓰비시중공업 주식회사를 상대로 객관적으로 권리를 사실상 행사할 수 없는 장애사유가 있었다고 봄이 타당하다고 한 사례\n\n        assistant : \n    ', 'label': '\n그러므로 상고를 모두 기각하고 상고비용은 패소자가 부담하도록 하여, 관여 대법관의 일치된 의견으로 주문과 같이 판결한다.\n상고이유(상고이유서 제출기간이 지난 후에 제출된 상고이유보충서의 기재는 상고이유를 보충하는 범위 내에서)를 판단한다

Map:   0%|          | 0/815 [00:00<?, ? examples/s]

dataset_dict:  {'prompt': '\n        system : 한국말로만 대답하고 최대한 간결하고 알기쉽게 정리해줘.\n        user : 사건의 title과 판시사항을 보고 판결 결과와 그 이유를 예측해줘\n        \n title:[재산세부과처분취소]〈오피스텔을 주택으로 취급하여 재산세를 부과할 수 있는지 문제된 사건〉 [공2024상,64]\n판시사항:오피스텔이 재산세 과세대상인 ‘주택’에 해당하는지 판단하는 기준\n\n        assistant : \n    ', 'label': '\n그러므로 상고를 기각하고 상고비용은 패소자가 부담하도록 하여, 관여 대법관의 일치된 의견으로 주문과 같이 판결한다.\n상고이유를 판단한다.\n1. 관련 법리\n지방세법 제105조, 제104조 제2호, 제6조 제4호, 건축법 제2조 제1항 제2호, 제2항 및 그 위임에 따른 건축법 시행령 제3조의5 [별표 1] 제14호는 재산세 과세대상인 ‘건축물’ 중 일반업무시설의 하나로 ‘오피스텔(업무를 주로 하며, 분양하거나 임대하는 구획 중 일부 구획에서 숙식을 할 수 있도록 한 건축물로서 국토교통부장관이 고시하는 기준에 적합한 것을 말한다)’을 명시하였고, 지방세법 제105조, 제104조 제3호, 구 주택법(2021. 12. 21. 법률 제18631호로 개정되기 전의 것, 이하 같다) 제2조 제1호는 재산세 과세대상인 ‘주택’을 ‘세대의 구성원이 장기간 독립된 주거생활을 할 수 있는 구조로 된 건축물의 전부 또는 일부 및 그 부속토지’로 정하면서, 재산세 과세대상인 토지와 건축물의 범위에서 주택은 제외한다고 규정하였다.\n구 지방세법 시행령(2021. 12. 31. 대통령령 제32293호로 개정되기 전의 것) 제119조는 ‘재산세의 과세대상 물건이 공부상 등재 현황과 사실상의 현황이 다른 경우에는 사실상 현황에 따라 재산세를 부과한다.’고 규정하였으므로, 재산세 과세대상인 ‘주택’에 해당하는지 여부는 특별한 사정이 없는 한 공부상의 용도에 관계없

Map:   0%|          | 0/91 [00:00<?, ? examples/s]

In [8]:
## ---  LoRA --- ##
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    # modules_to_save=["q_proj", "k_proj", "v_proj"],
)
model = get_peft_model(base_model, config)
model.print_trainable_parameters()

peft_model_id = f"MLP-KTLim/llama-3-Korean-Bllossom-8B_Lora"

trainable params: 4,718,592 || all params: 8,172,867,584 || trainable%: 0.0577


In [9]:
raise ValueError()

ValueError: 

In [ ]:
num_epochs = 30
learning_rate = 2e-5
optimizer = optim.Adam(
    model.parameters(),
    lr=learning_rate,
)
lr_scheduler = StepLR(optimizer, step_size=1, gamma=0.85)

In [ ]:
# accelerator.prepare(
#     train_dataloader, valid_dataloader, model, optimizer, lr_scheduler
# )

In [ ]:
def save_to_json(output_filename, epoch, train_epoch_loss, train_epoch_ppl, val_epoch_loss, val_epoch_ppl):
    metrics_data = {
        "epoch" : epoch,
        "train_epoch_loss": train_epoch_loss,
        "train_epoch_perplexity": train_epoch_ppl,
        "val_epoch_loss": val_epoch_loss,
        "val_epoch_perplexity": val_epoch_ppl
    }
    with open(output_filename, "a+") as f:
        json.dump(metrics_data, f)
        f.write('\n')


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
from tqdm import tqdm

device = "cuda"
model = model.to(device)
scaler = torch.cuda.amp.GradScaler()

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        # print(batch)
        with torch.autocast(device_type='cuda'):
            outputs = model(**batch)
            loss = outputs.loss
            total_loss += loss.detach().float()
        
        # accelerator.backward(loss)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        # loss.backward()
        # optimizer.step()
        # lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(valid_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )
    

    eval_epoch_loss = eval_loss / len(valid_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")
    save_to_json("./finetuning_train_result.json", epoch, float(train_epoch_loss), float(train_ppl.detach().cpu()), float(eval_epoch_loss), float(eval_ppl.detach().cpu()))
    

  0%|          | 0/815 [00:00<?, ?it/s]

100%|██████████| 91/91 [00:28<00:00,  3.19it/s]


epoch=0: train_ppl=tensor(16.3460, device='cuda:0') train_epoch_loss=tensor(2.7940, device='cuda:0') eval_ppl=tensor(9.6034, device='cuda:0') eval_epoch_loss=tensor(2.2621, device='cuda:0')


100%|██████████| 91/91 [00:28<00:00,  3.19it/s]


epoch=1: train_ppl=tensor(8.1980, device='cuda:0') train_epoch_loss=tensor(2.1039, device='cuda:0') eval_ppl=tensor(7.0572, device='cuda:0') eval_epoch_loss=tensor(1.9540, device='cuda:0')


...

epoch=27: train_ppl=tensor(1.0442, device='cuda:0') train_epoch_loss=tensor(0.0433, device='cuda:0') eval_ppl=tensor(1.0343, device='cuda:0') eval_epoch_loss=tensor(0.0337, device='cuda:0')


100%|██████████| 91/91 [00:28<00:00,  3.20it/s]


epoch=28: train_ppl=tensor(1.0561, device='cuda:0') train_epoch_loss=tensor(0.0546, device='cuda:0') eval_ppl=tensor(1.0475, device='cuda:0') eval_epoch_loss=tensor(0.0464, device='cuda:0')


100%|██████████| 91/91 [00:28<00:00,  3.20it/s]

epoch=29: train_ppl=tensor(1.0333, device='cuda:0') train_epoch_loss=tensor(0.0328, device='cuda:0') eval_ppl=tensor(1.0275, device='cuda:0') eval_epoch_loss=tensor(0.0271, device='cuda:0')


In [ ]:
model.save_pretrained(f"./{peft_model_id}/")

/home/eahc00/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
from peft import PeftModel

text_decoder_id = f"MLP-KTLim/llama-3-Korean-Bllossom-8B"
peft_model_id = f"MLP-KTLim/llama-3-Korean-Bllossom-8B_Lora"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# LoRA 적용 모델 불러오기
base_model = AutoModelForCausalLM.from_pretrained(
    text_decoder_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True, 
    quantization_config=bnb_config
    )


load_model = PeftModel.from_pretrained(base_model, peft_model_id)
loaded_state_dict = load_file("/home/eahc00/NLP/term_project/MLP-KTLim/llama-3-Korean-Bllossom-8B_Lora/adapter_model.safetensors")
load_model.load_state_dict(loaded_state_dict, strict=False)
tokenizer = AutoTokenizer.from_pretrained(text_decoder_id)

ds = get_dataset(split="valid", get_prompt_only=True)

load_model.to('cuda')

i = 1
inputs = tokenizer(ds[i]['prompt'], return_tensors="pt")
print(ds[i]['prompt'])
print(ds[i]['label'])

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



        system : 한국말로만 대답하고 최대한 간결하고 알기쉽게 정리해줘.
        user : 사건의 title과 판시사항을 보고 판결 결과와 그 이유를 예측해줘
        
 title:[손해배상(기)]〈일제 강제징용 노동자상을 제작·설치한 조각가 부부가 해당 노동자상의 모델이 일본인이라는 발언 등을 한 피고를 상대로 손해배상을 청구한 사건〉 [공2024상,97]
판시사항:[1] 민법상 불법행위가 되는 ‘명예훼손’의 의미 / 순수한 의견 표명 자체만으로 명예훼손이 성립하는지 여부(소극) 및 어떠한 표현이 사실의 적시인지 의견의 진술인지 판단하는 기준
[2] 명예훼손으로 인한 손해배상청구소송에서 적시된 사실의 허위성 및 위법성조각사유에 대한 증명책임의 분배
[3] 일제 강제징용 피해자를 상징하는 강제징용 노동자상을 제작한 조각가 부부 갑 등이 위 노동자상은 조선인이 아니라 일본인들을 모델로 만들었다는 발언들을 한 시의회 의원 을을 상대로 허위사실 적시에 의한 명예훼손 등을 주장하며 손해배상을 구한 사안에서, 위 발언들은 통상적인 어휘의 의미나 전후 문맥 등 전체적인 흐름, 사회평균인의 지식이나 경험 등을 고려하여 그 표현의 의미를 확정할 경우 사실의 적시가 아니라 의견의 표명이나 구체적인 정황 제시가 있는 의혹의 제기에 불과하여 명예훼손의 불법행위에 해당하지 않는다고 볼 여지가 많고, 위 발언들이 진실한 사실에 기초한 것이 아니라고 하더라도 을로서는 위 발언들을 행할 당시 그 내용이 진실이라고 믿을 만한 상당한 이유가 있었다고 볼 여지가 많은데도, 이와 달리 본 원심판결에 법리오해 등의 잘못이 있다고 한 사례

        assistant : 
    

그러므로 원심판결 중 피고 패소 부분을 파기하고, 이 부분 사건을 다시 심리·판단하도록 원심법원에 환송하기로 하여, 관여 대법관의 일치된 의견으로 주문과 같이 판결한다.
상고이유(상고이유서 제출기간이 지난 다음 제출된 서면의 기재는 상고이유를 보충하는 범위에서)를 판단한다.
1. 사

In [5]:
device = "cuda"

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = load_model.generate(input_ids=inputs["input_ids"], max_new_tokens=512)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:144783 for open-end generation.


['\n        system : 한국말로만 대답하고 최대한 간결하고 알기쉽게 정리해줘.\n        user : 사건의 title과 판시사항을 보고 판결 결과와 그 이유를 예측해줘\n        \n title:[손해배상(기)]〈일제 강제징용 노동자상을 제작·설치한 조각가 부부가 해당 노동자상의 모델이 일본인이라는 발언 등을 한 피고를 상대로 손해배상을 청구한 사건〉 [공2024상,97]\n판시사항:[1] 민법상 불법행위가 되는 ‘명예훼손’의 의미 / 순수한 의견 표명 자체만으로 명예훼손이 성립하는지 여부(소극) 및 어떠한 표현이 사실의 적시인지 의견의 진술인지 판단하는 기준\n[2] 명예훼손으로 인한 손해배상청구소송에서 적시된 사실의 허위성 및 위법성조각사유에 대한 증명책임의 분배\n[3] 일제 강제징용 피해자를 상징하는 강제징용 노동자상을 제작한 조각가 부부 갑 등이 위 노동자상은 조선인이 아니라 일본인들을 모델로 만들었다는 발언들을 한 시의회 의원 을을 상대로 허위사실 적시에 의한 명예훼손 등을 주장하며 손해배상을 구한 사안에서, 위 발언들은 통상적인 어휘의 의미나 전후 문맥 등 전체적인 흐름, 사회평균인의 지식이나 경험 등을 고려하여 그 표현의 의미를 확정할 경우 사실의 적시가 아니라 의견의 표명이나 구체적인 정황 제시가 있는 의혹의 제기에 불과하여 명예훼손의 불법행위에 해당하지 않는다고 볼 여지가 많고, 위 발언들이 진실한 사실에 기초한 것이 아니라고 하더라도 을로서는 위 발언들을 행할 당시 그 내용이 진실이라고 믿을 만한 상당한 이유가 있었다고 볼 여지가 많은데도, 이와 달리 본 원심판결에 법리오해 등의 잘못이 있다고 한 사례\n\n        assistant : \n    \n그러므로 원심판결 중 피고 패소 부분을 파기하고, 이 부분 사건을 다시 심리·판단하도록 원심법원에 환송하기로 하여, 관여 대법관의 일치된 의견으로 주문과 같이 판결한다.\n상고이유(상고이유서 제출기간이 지난 다음 제출된 서면의 기재는 상고이유를 보충하는 범위에서